Comapring the headlines and taglines with the text that we have extarcted from frames and audio.
For headines and taglines the text from audio will be given the first preference.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install fuzzywuzzy   # Documentation: https://github.com/seatgeek/fuzzywuzzy
from fuzzywuzzy import fuzz
import pandas as pd
import os, os.path

In [ ]:
# Use below lines to print and understand the extarcted text file
# Dataframe containing the text that we have obtained form the frames and audio
df_text=pd.read_excel("/content/drive/My Drive/Extracted_content/_Final.xlsx",header=0)
#df_text.head(2)

In [ ]:
# Use below lines to print the given headlines and taglines.
# Dataframe containing the reference headline and tagline.
df_head_tag=pd.read_excel('/content/drive/My Drive/Extracted_content/HeadLineTagLineMasterFile.xlsx',header=0)
#df_head_tag.head(2)

In [ ]:
def extract_videos(path, extensions):
  #Getting list of All videos 
  files = os.listdir(path)
  vdofiles = []
  vdopaths = []
  for file in files:
    file_arr = file.split('.')
    filename = file_arr[0]
    extension = file_arr[1]
    if extension in extensions:
      vdofiles.append(filename)
      vdopaths.append(os.path.join(path, file))
  return vdofiles, vdopaths

In [ ]:
df_text=pd.read_excel("/content/drive/My Drive/Extracted_content/_Final.xlsx",header=0)
df_head_tag=pd.read_excel('/content/drive/My Drive/Extracted_content/HeadLineTagLineMasterFile.xlsx',header=0)

# Adding columns for Headline Matching score deatils
df_head_tag['Headline_matched_Line_num']=""
df_head_tag['Headline_Score']=""
df_head_tag['Headline_text_Match']=""
df_head_tag['Headline_source']=""
# Adding columns for Tagline Matching score deatils
df_head_tag['Tagline_matched_Line_num']=""
df_head_tag['Tagline_Score']=""
df_head_tag['Tagline_text_Match']=""
df_head_tag['Tagline_source']=""

In [ ]:
def head_tag_score(Video_file_name,df_text,df_head_tag):
  # list to store the result of comparisons of given headline and tagline with frames and audio from respective videos
  Head_line_score=[]
  Tag_line_score=[]
  #to check the index of respective video in Head_Line_tag_line_excel file
  vid_index=[]
  #finding the matching score of headline and tagline
  for i in range(len(df_text['VDO-ID'])):
    if df_text['VDO-ID'][i]==Video_file_name:
      for j in range(len(df_head_tag["Ad Id (Video Id)"])):
        if df_head_tag["Ad Id (Video Id)"][j]==Video_file_name:
          text_line=df_text['Line_Text'][i]
          head_line=df_head_tag["Headline"][j]
          tg_line=df_head_tag["Tag line"][j]
          # Comparing the headline and text_line
          hd_line_score= fuzz.ratio(text_line.lower(),head_line.lower())
          Head_line_score.append([hd_line_score,i])
          # Comparing the tag_line and text_line
          tg_line_score= fuzz.ratio(text_line.lower(),tg_line.lower())
          Tag_line_score.append([tg_line_score,i])
          vid_index.append(j)
  #returning the maximum score of comparison of headline and tagline with (Ocr+Speech_to_text)
  #also returning the index of respective video in headlinetagline_excel_file
  return (max(Head_line_score),max(Tag_line_score),vid_index)


In [ ]:
def main():
  root_folder ="/content/drive/My Drive/vid"
  Content_folder="/content/drive/My Drive/Extracted_content"
  VdoAD = os.path.join(root_folder, "VDO-AD")
  extensions = ['mp4', 'mkv','avi']
  video_files, video_paths = extract_videos(VdoAD, extensions)
  for video_file_name in video_files:
    #calling the head_tag_score function on each video 
    head_score,tag_score,vid_index=head_tag_score(video_file_name,df_text,df_head_tag)
    #print(video_file_name)
    #print(head_score)
    #print(tag_score)
    #print(vid_index[0])
    #Writing the score obtained for Headline_matching in data frame against the respective video
    df_head_tag['Headline_matched_Line_num'][vid_index[0]]=df_text['Line_Num'][head_score[1]]  #line number of frame in which text is present
    df_head_tag['Headline_Score'][vid_index[0]]=head_score[0]  #Max score for Headline_match
    df_head_tag['Headline_text_Match'][vid_index[0]]= df_text['Line_Text'][head_score[1]]#text which has given the max matching score
    df_head_tag['Headline_source'][vid_index[0]]=df_text['Frame_name'][head_score[1]] # source of matched text(STT - Vid_name_Audio and OCR-vid_name_Frame_number)
    #Writing the score obtained for Tagline_matching in data frame against the respective video
    df_head_tag['Tagline_matched_Line_num'][vid_index[0]]=df_text["Line_Num"][tag_score[1]]
    df_head_tag['Tagline_Score'][vid_index[0]]=tag_score[0]
    df_head_tag['Tagline_text_Match'][vid_index[0]]= df_text['Line_Text'][tag_score[1]]
    df_head_tag['Tagline_source'][vid_index[0]]=df_text['Frame_name'][tag_score[1]]
    #saving the dataframe to a new file which will have the comparison score of headlines and tagline.
    # we are creating a new final file for score as we do not want to make any changes in the reference headline and tagline file.
  return(df_head_tag.to_excel('/content/drive/My Drive/Extracted_content/HeadLineTagLineMasterFile_Final.xlsx'))
  

In [ ]:
#calling main function
main()